In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("TempViews").getOrCreate()

In [ ]:
# In this example we are going to read data directly from a github repo.
url='https://raw.githubusercontent.com/datasets/five-thirty-eight-datasets/master/datasets/candy-power-ranking/data/candy-data.csv'

In [ ]:
# SparkFiles will allow you to resolves paths to files added through `SparkContext.addFile`

spark.sparkContext.addFile(url)
candy_data = spark.read.option('header', 'true').csv(SparkFiles.get("candy-data.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
candy_data.show()

+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|      competitorname|chocolate|fruity|caramel|peanutyalmondy|nougat|crispedricewafer|hard|bar|pluribus|sugarpercent|pricepercent|winpercent|
+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|           100 grand|        1|     0|      1|             0|     0|               1|   0|  1|       0|  0.73199999|  0.86000001| 66.971725|
|        3 musketeers|        1|     0|      0|             0|     1|               0|   0|  1|       0|  0.60399997|  0.51099998| 67.602936|
|            one dime|        0|     0|      0|             0|     0|               0|   0|  0|       0|       0.011|       0.116| 32.261086|
|         one quarter|        0|     0|      0|             0|     0|               0|   0|  0|       0|       0.011|  0.51099998| 46.116505|
|     

In [ ]:
# Create a temporary view. The temp view is a pre-requisite for Spark to allow you to read with sql.
candy_data.createOrReplaceTempView('candy')

In [ ]:
#using sql in Spark just requires you to call spark.sql(<sql>) and showing the data.
#here we are peeking at the data using SparkSQL
spark.sql("select * from candy limit 10").show()

+------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|    competitorname|chocolate|fruity|caramel|peanutyalmondy|nougat|crispedricewafer|hard|bar|pluribus|sugarpercent|pricepercent|winpercent|
+------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|         100 grand|        1|     0|      1|             0|     0|               1|   0|  1|       0|  0.73199999|  0.86000001| 66.971725|
|      3 musketeers|        1|     0|      0|             0|     1|               0|   0|  1|       0|  0.60399997|  0.51099998| 67.602936|
|          one dime|        0|     0|      0|             0|     0|               0|   0|  0|       0|       0.011|       0.116| 32.261086|
|       one quarter|        0|     0|      0|             0|     0|               0|   0|  0|       0|       0.011|  0.51099998| 46.116505|
|         air heads|

In [ ]:
# Get the the first 10 highest "winpercent" in desencending order.
spark.sql("SELECT * FROM candy ORDER BY winpercent DESC LIMIT 10").show()

+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|      competitorname|chocolate|fruity|caramel|peanutyalmondy|nougat|crispedricewafer|hard|bar|pluribus|sugarpercent|pricepercent|winpercent|
+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|reeses peanut but...|        1|     0|      0|             1|     0|               0|   0|  0|       0|  0.72000003|  0.65100002|  84.18029|
|   reeses miniatures|        1|     0|      0|             1|     0|               0|   0|  0|       0| 0.034000002|  0.27900001| 81.866257|
|                twix|        1|     0|      1|             0|     0|               1|   0|  1|       0|       0.546|  0.90600002| 81.642914|
|             kit kat|        1|     0|      0|             0|     0|               1|   0|  1|       0|  0.31299999|  0.51099998|   76.7686|
|     

In [ ]:
# Get the candies that have chocolate and caramel and order by the lowest sugar percentage.
spark.sql("SELECT * FROM candy WHERE chocolate==1 AND caramel==1 ORDER BY sugarpercent ASC").show()

+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|      competitorname|chocolate|fruity|caramel|peanutyalmondy|nougat|crispedricewafer|hard|bar|pluribus|sugarpercent|pricepercent|winpercent|
+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|           milk duds|        1|     0|      1|             0|     0|               0|   0|  0|       1|  0.30199999|  0.51099998| 55.064072|
|            snickers|        1|     0|      1|             1|     1|               0|   0|  1|       0|       0.546|  0.65100002| 76.673782|
|                twix|        1|     0|      1|             0|     0|               1|   0|  1|       0|       0.546|  0.90600002| 81.642914|
|           baby ruth|        1|     0|      1|             1|     1|               0|   0|  1|       0|  0.60399997|  0.76700002| 56.914547|
|     

In [ ]:
# Demonstrate how to write a SQL query on multiple lines.
query = """
(SELECT * FROM candy
WHERE chocolate==1 AND caramel==1
ORDER BY sugarpercent ASC)
"""
spark.sql(query).show()

+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|      competitorname|chocolate|fruity|caramel|peanutyalmondy|nougat|crispedricewafer|hard|bar|pluribus|sugarpercent|pricepercent|winpercent|
+--------------------+---------+------+-------+--------------+------+----------------+----+---+--------+------------+------------+----------+
|           milk duds|        1|     0|      1|             0|     0|               0|   0|  0|       1|  0.30199999|  0.51099998| 55.064072|
|            snickers|        1|     0|      1|             1|     1|               0|   0|  1|       0|       0.546|  0.65100002| 76.673782|
|                twix|        1|     0|      1|             0|     0|               1|   0|  1|       0|       0.546|  0.90600002| 81.642914|
|           baby ruth|        1|     0|      1|             1|     1|               0|   0|  1|       0|  0.60399997|  0.76700002| 56.914547|
|     